In [ ]:
pip install openai

In [6]:
import pandas as pd
import os
import re
#import openai
from openai import OpenAI

In [7]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

# Topic description

In [ ]:
df_map = pd.read_csv('data/map.txt', sep='\t')
df_map

In [ ]:
df_nodes = pd.read_csv('data/vertex.txt', sep='\t')
df_nodes = df_nodes[df_nodes.cluster.isin(df_map['id'])]
df_nodes

In [ ]:
df_terms = pd.read_csv('data/terms_all.tsv', sep='\t')
df_terms

In [ ]:
df_nodes = df_nodes.merge(df_terms[['term', 'occurrences', 'relevance score']], how='inner', left_on='name', right_on='term')
df_nodes

In [14]:
client = OpenAI(api_key='xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]

    response = client.chat.completions.create(
        messages=messages,
        model="gpt-3.5-turbo")

    return response.choices[0].message.content

In [ ]:
topics_name = pd.DataFrame()
topics_name

In [ ]:
for i in df_nodes[~df_nodes.cluster.isin(topics_name.cluster)].cluster.drop_duplicates().sort_values():
    print('\r'+str(i), end='')
    df_aux = df_nodes[df_nodes.cluster==i].sort_values('score', ascending=False).head(100)[['term']]
    df_aux = df_aux[['term']].drop_duplicates().reset_index(drop=True)
    prompt = "I will provide you with a list of terms extracted from the titles of academic works in the Humanities indexed in Dialnet, the bibliographic database for research in the Humanities and Social Sciences in Spanish. These terms specifically belong to a cluster of publications, meaning they share a thematic relation, and are also the most frequent and relevant terms of those publications. Therefore, from these terms, I need you to infer the research topic covered by the documents in this cluster. Please provide two things, separated by a newline (\n): first, the title of the topic, which is concise and perfectly encapsulates the topic, followed by a brief description explaining what the topic entails. Do not add any label or indication of which is the title or description; simply return the title first and then the description after the newline. Remember, these are academic texts in the Humanities from Spanish-speaking contexts, so the topic name should be appropriate to that. Please remember to return only the title and description of the topic, without anything else, and separate the title from the description with a single newline (\n) without introducing any other newlines to avoid confusion. Here is the list, with the terms separated by semicolons:"+'; '.join(df_aux['term'])
    response = get_completion(prompt)
    response = re.sub('(\n)+', '\n', response)

    try:
      topics_name_aux = pd.DataFrame({'Topic':response.split(sep='\n')[0], 'Description':response.split(sep='\n')[1]}, index=[0])
      topics_name_aux['cluster'] = i

      topics_name = pd.concat([topics_name, topics_name_aux]).reset_index(drop=True)
    except:
      pass

    topics_name.to_csv('data/topics_last.tsv', sep='\t', encoding='UTF-8', index=False)

In [ ]:
df_map = df_map.merge(topics_name, how='inner', left_on='id', right_on='cluster')
df_map

In [32]:
df_map.drop('cluster_y', axis=1, inplace=True)

In [33]:
df_map.to_csv('data/nodes_with_labels.tsv', sep='\t', encoding='UTF-8', index=False)